In [625]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spopt
import os
import plotly.graph_objects as go

In [626]:
# Creatin a linear regression function.
def average(inarray, avlen):
    
    arrlen = inarray.size
    
    outarray = np.zeros(int(arrlen//avlen))
    
    for i in range(0, int(arrlen//avlen)):
        for j in range(0, int(avlen)):
            outarray[i] += inarray[int(avlen)*i+j]/avlen
    
    return outarray

def linear(x, a, b):
    return a*x + b

In [627]:
#Constants
h = 4.135667696E-15 #eV s
c = 3E17 #nm/s 

In [ ]:
# Getting the path to the data
file_path= 'Data/'
files = os.listdir(file_path)

file_names_without_extension = [file.split('.TXT')[0] for file in files]

All_files = file_names_without_extension
All_files



In [ ]:
# ['1hr Sample 1_1812062U1',
#  '1hr Sample 2nd measurement_1812062U1',
#  '1hr Sample 3rd measurement_1812062U1',
#  '24hr Sample 1_1812062U1',
#  '24hr Sample 2nd measurement_1812062U1',
#  '24hr Sample 3rd measurement_1812062U1',
#  'CFO 2 1st measurement_1812062U1',
#  'CFO 2 2nd measurement_1812062U1',
#  'CFO 3 1st measurement_1812062U1',
#  'CFO 3 2nd measurement_1812062U1',
#  'CFO 4 1st measurement_1812062U1',
#  'CFO 4 2nd measurement_1812062U1',
#  'CFO 5 1st measurement_1812062U1',
#  'CFO 5 2nd measurement_1812062U1']

name = '24hr Sample 2nd measurement_1812062U1'  # ALways Change this part 
inputfilename = file_path + name + '.txt'
plotfilename = file_path + name  + '.png'

inputfilename

In [630]:
# Reading in the data in txt format
#Check for column separator (sep) and decimal separator (decimal)
data = pd.read_csv(inputfilename, sep=';', decimal='.', header=8)
data.columns = ['wavelength', 'spectrum', 'dark', 'reference', 'reflectance']

In [ ]:
data.head()

In [ ]:
# see the wavelength below 200  

data.loc[data['wavelength'] > 200]

In [ ]:
#The following two arrays contain the measured wavelenght and reflectance values.
#The latter are scaled to 0 to 1.
wavelength = np.asarray(data['wavelength'].tolist())
reflectance = np.asarray(data['reflectance'].tolist())/100
print(f"Wavelength: {wavelength}\nReflectance: {reflectance}")

In [ ]:
#All measurements below 200 nm are discarded as there is no light below that anyway.
cutindex = np.argmax(wavelength>200.)
wavelength = wavelength[cutindex:]
reflectance = reflectance[cutindex:]
print(f"Wavelength: {wavelength}\nReflectance: {reflectance}")

In [ ]:
#All data points with reflectance above 100% or 1 are removed.
#Comment the following three lines out if not wanted.
greaterone = np.argwhere(reflectance > 1.0)
wavelength = np.delete(wavelength, greaterone)
reflectance = np.delete(reflectance, greaterone)
print(f"Wavelength: {wavelength}\nReflectance: {reflectance}")

In [ ]:
#In order to remove noise subsequent points are averaged. 
#Aver specifies how many neighboring points. Aver = 1 means no change.
#
aver = 2
wave_ave = average(wavelength,aver)
refl_ave = average(reflectance,aver)
print(f"Wave_ave: {wave_ave}\nRefl_ave: {refl_ave}")

In [ ]:
#A plot of the measured data and the averaged data
# Create the figure and specify the axes
fig = go.Figure()

# Add the scatter plot for the measured data
fig.add_trace(go.Scatter(x=wavelength, y=reflectance*100, mode='markers', marker=dict(color='blue'), name='Measured Data'))

# Set the layout for the plot
fig.update_layout(
    title="A plot of the measured data and the averaged data " + name,
    xaxis_title="Wavelength (nm)",
    yaxis_title="Diffuse Reflectance (%)",
    xaxis=dict(range=[400, 1050], autorange=False),
    yaxis=dict(range=[0, 100], autorange=False, type="linear")
)

# Show the plot
fig.show()


In [ ]:
#The reflectance is used to calculate the Tauc-Plot data.
#Always check that the exponent in the y formula is correct for your case.
#
energy = np.flip(h*c*(1./wave_ave))
alpha = np.flip((1.0 - refl_ave)**2/(2*refl_ave))
print(f"Energy: {energy}\nAlpha: {alpha}")

In [ ]:
# determine the size of the energ and alpha values

size = len(energy)
print(f"Energy: {energy}\nAlpha: {alpha}")
print(f"Size: {size}")

In [640]:
# # Filter invalid values for sqrt calculation
# valid_indices = np.where(alpha * energy > 0)
# energy_valid = energy[valid_indices]
# alpha_valid = alpha[valid_indices]
# y = (alpha_valid * energy_valid) ** (1. / 2)

In [641]:
# #y = (alpha*energy)**(2)
y = np.sqrt(np.abs(alpha * energy))
# #The derivate of y sometimes is needed for error checking
# yprime = np.gradient(y, energy)
# yprime

In [642]:
# # Ensure the lengths of energy_valid and y match
# assert len(energy_valid) == len(y), "Lengths of energy and y do not match"
assert len(energy) == len(y)

## Linear regression

In [643]:
#The next section fits a linear function to the specific start and stop energy range.
#The bandgap and its statistical error are printed in the console.
#

# Define the percentage thresholds for start and stop fit values
start_fit_value = 1.5
stop_fit_value = 2.1

In [644]:
start_fit = np.argmax(energy>start_fit_value)
stop_fit = np.argmax(energy>stop_fit_value)-1
fit_energy = energy[start_fit:stop_fit]
fit_y = y[start_fit:stop_fit]

In [645]:
# Calculate the error on the fit parameters
popt1, pcov1 = spopt.curve_fit(linear, fit_energy, fit_y)
perr1 = np.sqrt(np.diag(pcov1))

In [ ]:
# Calculate and print bandgap
bandgap = -popt1[1] / popt1[0]
bandgap_error = np.sqrt((perr1[1] / popt1[0]) ** 2 + (perr1[0] * popt1[1] / popt1[0] ** 2) ** 2)
print(f'Bandgap is equal to {bandgap:.2f} +/- {bandgap_error:.2f} eV.')

In [647]:
# #WHAT IS THE NEXT LIONE FOr
# print('Bandgap is equal ' + str(-popt1[1]/popt1[0]) + ' +/- ' + str(np.sqrt((perr1[1]/popt1[0])**2+(perr1[0]*popt1[1]/popt1[0]**2)**2)) + 'eV.')


In [ ]:
# Plot Tauc plot and linear fit using Plotly
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=energy,
    y=y,
    mode='markers',
    name='Tauc Plot Data'
))

fig2.add_trace(go.Scatter(
    x=fit_energy,
    y=linear(fit_energy, *popt1),
    mode='lines',
    name='Linear Fit',
    line=dict(color='red')
))

fig2.update_layout(
    xaxis_title='Energy (eV)',
    yaxis_title='(αhν)^(1/2) (eV^(1/2))',
    xaxis=dict(range=[1.1, 3.0]),
    yaxis=dict(range=[0, 5])
)

fig2.show()